# Model Validation and Data Leakage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, make_scorer

from useful_functions import *

%load_ext autoreload
%autoreload 2

## Objectives

- explain the bias-variance tradeoff and the correlative notions of underfit and overfit models
- describe a train-test split and explain its purpose in the context of predictive statistics / machine learning
- explain the algorithm of cross-validation
- use best practices for building non-leaky workflows
- repair leaky workflows

## Motivation

At this point, we have seen different ways to create models from our data through different linear regression techniques. That's good. But when it comes to measuring model performance, we also want to make sure that our models are ready to predict on data that they haven't seen yet.

Usually, when our model is ready to be used in the "real world" we refer to this as putting our model into **production** or **deploying** our model. The data points for which it will make predictions will be data *it has never seen before*, as opposed to the data points that were used to train the model.

This is where ***model validation*** techniques come in, namely, to ensure our model can *generalize* to data it hasn't directly seen before.

As a way into a discussion of these techniques let's say a word about the **bias-variance tradeoff**.

## The Bias-Variance Tradeoff

We can break up how the model makes mistakes (the error) by saying there are three parts:

- Error inherent in the data (noise): **irreducible error**
- Error from not capturing signal (too simple): **bias**
- Error from "modeling noise", i.e. capturing patterns in the data that don't generalize well (too complex): **variance**

We can summarize this in an equation for the _mean squared error_ (MSE):

$MSE = Bias(\hat{y})^2 + Var(\hat{y}) + \sigma^2$

![optimal](images/optimal_bias_variance.png)
http://scott.fortmann-roe.com/docs/BiasVariance.html

### Bias

**High-bias** algorithms tend to be less complex, with simple or rigid underlying structure.

![](images/noisy-sine-linear.png)

+ They train models that are consistent, but inaccurate on average.
+ These include linear or parametric algorithms such as regression and naive Bayes.
+ The following sorts of difficulties could lead to high bias:
  - We did not include the correct predictors
  - We did not take interactions into account
  - We missed a non-linear (polynomial) relationship

      
High-bias models are generally **underfit**: The models have not picked up enough of the signal in the data. And so even though they may be consistent, they don't perform particularly well on the initial data, and so they will be consistently inaccurate.

### Variance

On the other hand, **high-variance** algorithms tend to be more complex, with flexible underlying structure.

![](images/noisy-sine-decision-tree.png)

+ They train models that are accurate on average, but inconsistent.
+ These include non-linear or non-parametric algorithms such as decision trees and nearest-neighbor models.
+ The following sorts of difficulties could lead to high variance:
  - We included an unreasonably large number of predictors;
  - We created new features by squaring and cubing each feature.

High variance models are **overfit**: The models have picked up on the noise as well as the signal in the data. And so even though they may perform well on the initial data, they will be inconsistently accurate on new data.

![](images/target.png)

### Balancing Bias and Variance

While we build our models, we have to keep this relationship in mind.  If we build complex models, we risk overfitting our models.  Their predictions will vary greatly when introduced to new data.  If our models are too simple, the predictions as a whole will be inaccurate.   

![](images/noisy-sine-third-order-polynomial.png)

The goal is to build a model with enough complexity to be accurate, but not too much complexity to be erratic.

## 🧠 Knowledge Check

![which_model](images/which_model_is_better_2.png)

## Train-Test Split

It is hard to know if your model is too simple or complex by just using it on training data.

We can _hold out_ part of our training sample, use it as a test sample, and then use it to monitor our prediction error.

This allows us to evaluate whether our model has the right balance of bias/variance. 

<img src='images/testtrainsplit.png' width =550 />

* **training set** —a subset to train a model.
* **test set**—a subset to test the trained model.

## Is the Model Overfitting or Underfitting?

If our model is not performing well on the training  data, we are probably underfitting it.  

To know if our  model is overfitting the data, we need  to test our model on unseen data. 
We then measure our performance on the unseen data. 

If the model performs significantly worse on the  unseen data, it is probably  overfitting the data.

<img src='https://developers.google.com/machine-learning/crash-course/images/WorkflowWithTestSet.svg' width=500/>

## Practice Exercises: Name that Model!

Consider the following scenarios and describe them according to bias and variance. There are four possibilities:

- a. The model has low bias and high variance.
- b. The model has high bias and low variance.
- c. The model has both low bias and low variance.
- d. The model has both high bias and high variance.

**Scenario 1**: The model has a low RMSE on training and a low RMSE on test.
<details>
    <summary> Answer
    </summary>
    b. The model has both low bias and low variance.
    </details>

**Scenario 2**: The model has a high $R^2$ on the training set, but a low $R^2$ on the test.
<details>
    <summary> Answer
    </summary>
    a. The model has low bias and high variance.
    </details>

**Scenario 3**: The model performs well on data it is fit on and well on data it has not seen.
<details>
    <summary> Answer
    </summary>
    c. The model has both low bias and low variance.
    </details>
  

**Scenario 4**: The model has a low $R^2$ on training but high on the test set.
<details>
    <summary> Answer
    </summary>
    d. The model has both high bias and high variance.
    </details>

**Scenario 5**: The model leaves out many of the meaningful predictors, but is consistent across samples.
<details>
    <summary> Answer
    </summary>
    b. The model has high bias and low variance.
    </details>

**Scenario 6**: The model is highly sensitive to random noise in the training set.
<details>
    <summary> Answer
    </summary>
    a. The model has low bias and high variance.
    </details>

## Should You Ever Fit on Your Test Set?  

![no](https://media.giphy.com/media/d10dMmzqCYqQ0/giphy.gif)

**Never fit on test data.** If you are seeing surprisingly good results on your evaluation metrics, it might be a sign that you are accidentally training on the test set.

## Train-Test Split Example

In [ ]:
data = load_diabetes()

print(data.DESCR)

In [ ]:
df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

In [ ]:
df.describe()

In [ ]:
df.corr()['target'].map(abs).sort_values(ascending=False)

In [ ]:
X = df[['bmi', 's5']]
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [ ]:
display(X_train.head())
display(X_test.head())

In [ ]:
y_train

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(X_train.shape[0] == y_train.shape[0])
print(X_test.shape[0] == y_test.shape[0])

In [ ]:
# Instanstiate your linear regression object
lr = LinearRegression()

In [ ]:
# fit the model on the training set
lr.fit(X_train, y_train)

In [ ]:
# Check the R^2 of the training data
lr.score(X_train, y_train)

In [ ]:
lr.coef_

A .440 R-squared reflects a model that explains about half of the total variance in the data. 

## Now check performance on test data

Next, we test how well the model performs on the unseen test data. Remember, we do not fit the model again. The model has calculated the optimal parameters learning from the training set.  

In [ ]:
lr.score(X_test, y_test)

## 🧠 Knowledge Check

How would you describe the bias of the model based on the above training $R^2$?

<details>
    <summary> Answer
    </summary>
    The difference between the train and test scores is low. High bias
    </details>

What does that indicate about variance? somewhat low variance

## Same Procedure with a Polynomial Model

In [ ]:
poly_2 = PolynomialFeatures(4)

X_poly = pd.DataFrame(
            poly_2.fit_transform(df.drop('target', axis=1))
                      )

y = df.target

In [ ]:
X_poly

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y,
                                                    test_size=0.25,
                                                    random_state=42)
lr_poly = LinearRegression()

# Always fit on the training set
lr_poly.fit(X_train, y_train)

lr_poly.score(X_train, y_train)

In [ ]:
lr_poly.score(X_test, y_test)

In [ ]:
len(lr_poly.coef_)

In [ ]:
mean_squared_error(y_train, lr_poly.predict(X_train), squared=False)

In [ ]:
mean_squared_error(y_test, lr_poly.predict(X_test), squared=False)

## Data Leakage

[This post about scaling and data leakage](https://datascience.stackexchange.com/questions/38395/standardscaler-before-and-after-splitting-data) explains that if you are going to scale your data, you should only train your scaler on the training data to prevent data leakage.  

We have encountered the idea of splitting our data into two, *training* our model on one bit and then *testing* it on the other.

The goal is to have an unbiased assessment of our model, and so we want to make sure that nothing about our test data sneaks into the training run of the model.

### A Mistake

Now consider the following workflow:

In [ ]:
df = pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
               pd.Series(data.target, name='target')], axis=1)

df.head()

In [ ]:
X, y = load_diabetes(return_X_y=True)

In [ ]:
X

In [ ]:
y

In [ ]:
ss = StandardScaler().fit(X)
X_scld = ss.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scld, y, random_state=42)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.coef_, lr.intercept_)

Well we fit the model only to our training data. Looks like we've done everything right, right?

It's important to understand that the answer here is a resounding "NO". It's true that we didn't directly fit our model to our test data. But the trouble is that we fit our scaler **to the whole dataset**. That is, records in the test data are contributing to calculations of column means and standard deviations, and so, surreptitiously, information about our test set is sneaking into the training run of the model after all!

To correct our mistake, we'll make sure to perform our train-test split **first**:

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=42)

In [ ]:
ss2 = StandardScaler().fit(X_train2)
X_train2_scld = ss2.transform(X_train2)
X_test2_scld = ss2.transform(X_test2)

In [ ]:
lr2 = LinearRegression()
lr2.fit(X_train2_scld, y_train2)
print(lr2.coef_, lr2.intercept_)

Note that our model coefficients are slightly different from what they were before.

#### Error Comparison

It's worth pointing out that, **for linear models**, there is **no** difference in modeling error:

In [ ]:
y_test_hat = lr.predict(X_test)
mse = mean_squared_error(y_test, y_test_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

In [ ]:
y_test2_hat = lr2.predict(X_test2_scld)
mse = mean_squared_error(y_test2, y_test2_hat)
print(f"Our test RMSE for this model is {round(np.sqrt(mse), 2)}.")

This will **not** be true for other sorts of models that use different loss functions.

### Preprocessing

In general all preprocessing steps are subject to the same dangers here. Consider the preprocessing step of one-hot-encoding:

In [ ]:
gun_poll = pd.read_csv('data/guns-polls.csv')

gun_poll.head()

In [ ]:
gun_poll['Pollster'].value_counts()

Now if I were to fit a one-hot encoder to the whole `Pollster` column here, the encoder would learn all the categories. But I need to prepare myself for the real-world possibility that unfamiliar categories may show up in future records. Let's explore this.

In [ ]:
# First I'll do a split
X_train, X_test = train_test_split(gun_poll, random_state=42)

In [ ]:
X_train['Pollster'].value_counts()

In [ ]:
X_test['Pollster'].value_counts()

Let's suppose now that I fit a `OneHotEncoder` to the `Pollster` column in my training data.

#### Exercise

Fit an encoder to the `Pollster` column of the training data and then check to see which categories are represented.

In [ ]:
dummy_col = X_train[['Pollster']]
ohe = OneHotEncoder(sparse=False)
ohe.fit(dummy_col)

ohe.get_feature_names()

<details>
    <summary>Answer</summary>
<code>to_be_dummied = X_train[['Pollster']]
ohe = OneHotEncoder()
ohe.fit(to_be_dummied)
## So what categories do we have?
ohe.get_feature_names()</code>
</details>

We'll want to transform both train and test after we've fitted the encoder to the train.

In [ ]:
ohe.transform(dummy_col)

In [ ]:
ohe.transform(X_test[['Pollster']])

There are categories in the testing data that don't appear in the training data! What should 
we do about that?

### Approaches

- **Strategy 1**: Divide up the categories proportionally when we do our train_test_split. If we're using `sklearn`'s tool, that means taking advantage of the `stratify` parameter:

In [ ]:
new_X_train, new_X_test = train_test_split(gun_poll, 
                                           stratify=gun_poll['Pollster'], random_state=42)

Unfortunately, in this case, we can't use this since some categories have only a single member.

- **Strategy 2**: Drop the categories with very few representatives.

In the present case, let's try dropping the single-member categories.

In [ ]:
vc = gun_poll['Pollster'].value_counts()
to_drop = list(vc[vc == 1].index)
to_drop

In [ ]:
subset = gun_poll.loc[~gun_poll['Pollster'].isin(to_drop)]

In [ ]:
subset['Pollster'].value_counts()

In [ ]:
vc_only_1 = vc[vc==1]
bad_cols = vc_only_1.index
bad_cols

In [ ]:
gun_poll['Pollster'] = gun_poll['Pollster'].map(lambda x: np.nan if x in bad_cols else x)
gun_poll = gun_poll.dropna()

gun_poll['Pollster'].value_counts()

We could now split this carefully so that new categories don't show up in the testing data. In fact, now we can try the stratified split:

In [ ]:
X_train3, X_test3 = train_test_split(gun_poll,
                                     stratify=gun_poll['Pollster'],
                                     test_size=0.3,
                                     random_state=42)

X_train3['Pollster'].value_counts()

In [ ]:
X_test3['Pollster'].value_counts()

Now every category that appears in the test data appears also in the training data.

- **Strategy 3**: Adjust the settings on the one-hot-encoder.

For `sklearn`'s tool, we'll tweak the `handle_unknown` parameter:

In [ ]:
ohe2 = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohe2.fit(dummy_col)
ohe2.transform(dummy_col)

In [ ]:
ohe2.transform(X_test[['Pollster']])

#### Exericse

Fit a new encoder to our training data column that won't break when we try to use it to transform the test data. And then use the encoder to transform both train and test.

<details>
    <summary>Answer</summary>
<code>ohe2 = OneHotEncoder(handle_unknown='ignore')
ohe2.fit(to_be_dummied)
test_to_be_dummied = X_test[['Pollster']]
ohe2.transform(to_be_dummied)
ohe2.transform(test_to_be_dummied)</code>
</details>

## Level Up: $k$-Fold Cross-Validation: Even More Rigorous Validation  

Our goal of using a test set is to simulate what happens when our model attempts predictions on data it's never seen before. But it's possible that our model would *by chance* perform well on the test set.

This is where we could use a more rigorous validation method and turn to **$k$-fold cross-validation**.

![kfolds](images/k_folds.png)

[image via sklearn](https://scikit-learn.org/stable/modules/cross_validation.html)

In this process, we split the dataset into a train set and holdout test sets like usual by performing a shuffling train-test split on the train set.  

We then do $k$-number of _folds_ of the training data. This means we divide the training set into different sections or folds. We then take turns on using each fold as a **validation set** (or **dev set**) and train on the larger fraction. Then we calculate a validation score from the validation set the model has never seen. We repeat this process until each fold has served as a validation set.

This process allows us to try out training our model and check to see if it is likely to overfit or underfit without touching the holdout test data set.

If we think the model is looking good according to our cross-validation using the training data, we retrain the model using all of the training data. Then we can do one final evaluation using the test data. 

It's important that we hold onto our test data until the end and refrain from making adjustments to the model based on the test results.

In [ ]:
## Example

X = df.drop('target', axis=1)
y = df['target']

In [ ]:
# Let's create our holdout test
X_train, X_test, y_train, y_test = train_test_split(
                                                X,
                                                y,
                                                test_size=0.2,
                                                random_state=42
)

In [ ]:
### Simple Model

model_simple = LinearRegression()
scores_simple = None
print(f"""train scores: {scores_simple['train_score']},
      test scores: {scores_simple['test_score']}""")

In [ ]:
# Mean train r_2
None

In [ ]:
# Mean validation r_2
None

In [ ]:
# Fit on all the training data
model_simple.fit(X_train, y_train)
model_simple.score(X_train, y_train)

### More Complex Model

In [ ]:
# Test out our polynomial model
poly_3 = PolynomialFeatures(3)
X_poly3 = poly_3.fit_transform(X_train)

model_poly3 = LinearRegression()
scores_complex3 = cross_validate(
                        model_poly3, X_poly3, y_train, cv=5, 
                        return_train_score=True
)
print(f"""train scores: {scores_complex3['train_score']},
      test scores: {scores_complex3['test_score']}""")

In [ ]:
# Mean train r_2
np.mean(scores_complex3['train_score']), np.std(scores_complex3['train_score']) 

In [ ]:
# Mean test r_2
np.mean(scores_complex3['test_score']), np.std(scores_complex3['test_score'])

In [ ]:
# Fit on all the training data
model_poly3.fit(X_poly3, y_train)
model_poly3.score(X_poly3, y_train)

### Medium-Complexity Model

In [ ]:
# Test out our polynomial model
poly_2 = PolynomialFeatures(2)
X_poly2 = poly_2.fit_transform(X_train)

model_poly2 = LinearRegression()
scores_complex2 = cross_validate(
                        model_poly2, X_poly2, y_train, cv=5, 
                        return_train_score=True
)
print(f"""train scores: {scores_complex2['train_score']},
      test scores: {scores_complex2['test_score']}""")

In [ ]:
# Mean train r_2
np.mean(scores_complex2['train_score']), np.std(scores_complex2['train_score']) 

In [ ]:
# Mean test r_2
np.mean(scores_complex2['test_score']), np.std(scores_complex2['test_score'])

model_poly2.fit(X_poly2, y_train)
model_poly2.score(X_poly2, y_train)

### Checking Our Models Against the Holdout Test Set

Once we have an acceptable model, we train our model on the entire training set and score on the test to validate.

In [ ]:
best_model = None

In [ ]:
# Remember that we have to transform X_test in the same way
None

## Leakage into Validation Data

If we employ cross-validation, then our training data points will be serving both for training and for validation. So there's a sense in which we can't help but let some information about our validation data sneak into the model.

But strictly speaking, cross-validation means building *multiple* models, and we still want each to be blind to its validation set.

The dangers of data leakage, therefore, are still very much real in the case of validation data. And they are often more subtle as well. Consider the following line of code:

In [ ]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

# Using our scaled training data

cv_results = cross_validate(estimator=LinearRegression(),
                X=X_train2_scld,
                y=y_train2,
                scoring=mse,
                return_estimator=True)

We've built five models here, and none of them saw any points from the test data, so we have no leaks, right?

Wrong! We fit the `StandardScaler` to the whole training set, which means that information about *every* fold will affect every cross-validation. A better practice here would be to split our data into its cross-validation folds *first*. Then we can fit the scaler to only the training folds for each cross-validation.

Of course, the more preprocessing steps we have, the more tedious it becomes to do this work! For such tasks it is often greatly beneficial to take advantage of `sklearn`'s `Pipeline`s, which we'll have more to say about later.

For now, let's see if we can fix our leaky cross-validation scorer.

#### Exercise

Go back to `X_train2` and do the following:

- Split it into five validation folds (Use `KFold()`).
- For each split:
- (i) fit a `StandardScaler` to the four-fold chunk and transform all data points with it.
- (ii) fit a `LinearRegression` to the four-fold chunk and print out the value of the first coefficient.

<details>
    <summary>Answer</summary>

<code>for train_ind, val_ind in KFold().split(X_train2):
    train = X_train2[train_ind, :]
    val = X_train2[val_ind, :]
    target_train = y_train2[train_ind]
    target_val = y_train2[val_ind]
    ss = StandardScaler().fit(train)
    train_scld = ss.transform(train)
    val_scld = ss.transform(val)
    lr = LinearRegression().fit(train_scld, target_train)
    print(lr.coef_[0])</code>
</details>